In [0]:
#import the dataset
data = spark.read.csv("dbfs:/FileStore/tables/automotive_sales-1.csv", inferSchema = True, header = True, sep = ",").cache()

In [0]:
#check the data type of each feature
data.printSchema()

root
-- cust: integer (nullable = true)
-- creditorcash: string (nullable = true)
-- gender: string (nullable = true)
-- product: string (nullable = true)
-- age: integer (nullable = true)
-- purchase_date: string (nullable = true)

In [0]:
#change the new_purchase_date in case it's not date type yet
from pyspark.sql.functions import unix_timestamp, from_unixtime, to_date
data = data.withColumn('new_purchase_date', to_date(unix_timestamp('purchase_date', 'yyyy/MM/dd').cast("timestamp"))).drop('purchase_date')

In [0]:
#sort by cust and purchase date
data = data.orderBy(["cust", "new_purchase_date"], ascending=True)

In [0]:
#calculating the days interval between purchases for each customer which will be our target variable
from pyspark.sql.functions import *
from pyspark.sql.window import Window
data = data.withColumn("timeInterval", datediff(data.new_purchase_date, lag(data.new_purchase_date, 1)
    .over(Window.partitionBy("cust")
    .orderBy("new_purchase_date"))))
data = data.na.fill({'timeInterval': 0})

In [0]:
#create flag of the i-th number of purchase (1st time, 2nd time, etc.) as a new feature
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, dense_rank
from pyspark.sql.functions import unix_timestamp, from_unixtime, to_date

window = Window.partitionBy(data['cust']).orderBy(data['new_purchase_date'])
data = data.select('*', rank().over(window).alias('flag'))

In [0]:
#evaluate the correlation of numerical features with the target variable
import six

corr_cols = ["age"]
for i in corr_cols:
    if not( isinstance(data.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to timeInterval for ", i, data.stat.corr('timeInterval',i))

Correlation to timeInterval for age 0.24728588474479177

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder
stages = []
cat_cols = ["gender", "product", "flag"]

for cat_col in cat_cols:
  indexer = StringIndexer(inputCol = cat_col, outputCol = cat_col + "_index", stringOrderType = "alphabetAsc")
  encoder = OneHotEncoder(inputCols = [indexer.getOutputCol()], outputCols = [cat_col + "_vec"], dropLast = True)
  stages += [indexer, encoder]
  
pipeline = Pipeline(stages=stages)
data2 = pipeline.fit(data).transform(data)

In [0]:
#split the date into training and testing with set up seed for consistency
(train, test) = data2.randomSplit([0.8, 0.2], seed=123)
train.cache()
test.cache()

Out[12]: DataFrame[cust: int, creditorcash: string, gender: string, product: string, age: int, new_purchase_date: date, timeInterval: int, flag: int, gender_index: double, gender_vec: vector, product_index: double, product_vec: vector, flag_index: double, flag_vec: vector]

In [0]:
# TODO: Build a better Regression pipeline
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler

assemblerInputs = ["gender_vec", "product_vec", "flag_vec", "age"]

vectorAssembler = VectorAssembler(
  inputCols = assemblerInputs, 
  outputCol="features")

rfr = (RandomForestRegressor()
      .setLabelCol("timeInterval") # The column of our label
      .setSeed(123)        # Some seed value for consistency
      .setNumTrees(10)   # A guess at the number of trees
      .setMaxDepth(30)    # A guess at the depth of each tree
)

pipelinerfr = Pipeline().setStages([
  vectorAssembler,
  rfr
])

pipelineModelrfr = pipelinerfr.fit(train)

predictionsDFrfr = pipelineModelrfr.transform(test)

from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator().setLabelCol("timeInterval")

rmserfr = evaluator.evaluate(predictionsDFrfr)

print("Test RMSE Random Forest Regressor = %f" % rmserfr)

Test RMSE Random Forest Regressor = 240.711339

In [0]:
#compare the RMSE with the average to get the sense of how well the model works as a simple analysis
from pyspark.sql.functions import col, avg
data.agg(avg(col("timeInterval"))).show()

+-----------------+
avg(timeInterval)|
+-----------------+
 773.08|
+-----------------+